# **Steam data analysis**
### Data from Steam API and SteamSpy, dataset by Craig Kelly <br/> Refactored and appended by Adriyan Bevz

In [1]:
import pandas as pd
import altair as alt
import numpy as np

In [2]:
alt.renderers.enable('default')
alt.data_transformers.enable('json')

DataTransformerRegistry.enable('json')

## **Load the datasets:**

In [3]:
df = pd.read_csv('refactored.csv')

## **Overview the columns:**

In [4]:
fr = df.iloc[0]

for c in df.columns:
    print(c + ": " + str(fr[c]))

Unnamed: 0: 0
Unnamed: 0.1: 0
QueryID: 10
ResponseID: 10
QueryName: Counter-Strike
ResponseName: Counter-Strike
ReleaseDate: Nov 1 2000
RequiredAge: 0
DLCCount: 0
Metacritic: 88
RecommendationCount: 68991
SteamSpyOwners: 13033334
SteamSpyPlayersEstimate: 9140731
IsFree: False
PlatformWindows: True
PlatformLinux: True
PlatformMac: True
PriceCurrency: USD
PriceInitial: 9.99
PriceFinal: 9.99
Reviews:  
SupportedLanguages: English French German Italian Spanish Simplified Chinese Traditional Chinese Korean
Categories: Multiplayer
Genres: Action
ConcurrentPlayers: 16118
DaysSinceRelease: 6998


In [5]:
# https://gist.github.com/jlln/338b4b0b55bd6984f88

def splitDataFrameList(target_df, target_column, separator):
    ''' df = dataframe to split,
    target_column = the column containing the values to split
    separator = the symbol used to perform the split
    returns: a dataframe with each entry for the target column separated, with each element moved into a new row. 
    The values in the other columns are duplicated across the newly divided rows.
    '''
    def splitListToRows(row, row_accumulator, target_column, separator):
        if row[target_column] == None:
            return
        split_row = str(row[target_column]).split(separator)
        for s in split_row:
            new_row = row.to_dict()
            new_row[target_column] = s
            row_accumulator.append(new_row)
    new_rows = []
    target_df.apply(splitListToRows, axis=1, args = (new_rows, target_column, separator))
    new_df = pd.DataFrame(new_rows)
    return new_df

## **Set scale for the charts:**

In [6]:
SCALE = 1

## **Refactor the columns:**

In [7]:
category_exploded_df = splitDataFrameList(df, 'Categories', ',')
genre_exploded_df = splitDataFrameList(df, 'Genres', ',')
category_genre_exploded_df = splitDataFrameList(category_exploded_df, 'Genres', ',')

## **Game count by category and genre:**

In [110]:
category_genre_count_chart = alt.Chart(category_genre_exploded_df).transform_filter(
    (alt.datum.Genres != 'nan') & (alt.datum.Categories != 'nan')
).mark_point(color='Indigo').encode(
    x=alt.X('Genres:N', axis=alt.Axis(labelAngle=-45)),
    y=alt.Y('Categories:N'),
    size=alt.Size('count()', scale=alt.Scale(range=[1, 1200 * SCALE])),
    tooltip=alt.Tooltip(['count()']),
).properties(
    width=500 * SCALE,
    height=250 * SCALE
)

category_count_bars = alt.Chart(category_exploded_df).transform_filter(
    alt.datum.Categories != 'nan'
).mark_bar(color='Indigo').encode(
    x=alt.X('count()'),
    y=alt.Y('Categories:N', axis=None),
    tooltip=alt.Tooltip(['count()'])
).properties(
    width=200 * SCALE,
    height=250 * SCALE
)

genre_count_bars = alt.Chart(genre_exploded_df).transform_filter(
    alt.datum.Genres != 'nan'
).mark_bar(color='Indigo').encode(
    x=alt.X('Genres:N', axis=None),
    y=alt.Y('count()'),
    tooltip=alt.Tooltip(['count()'])
).properties(
    width=500 * SCALE,
    height=200 * SCALE
)

(genre_count_bars & (category_genre_count_chart | category_count_bars)).properties(
    title='Game count by category and genre',
).configure_title(
    fontSize=20,
)

alt.VConcatChart(...)

### Conclusion:
Unsurprisingly, singleplayer indie games are the dominating intersection - more than half the games are marked this way.

## **Owner count by category and genre:**

In [109]:
category_genre_owner_chart = alt.Chart(category_genre_exploded_df).transform_filter(
    (alt.datum.Genres != 'nan') & (alt.datum.Categories != 'nan')
).mark_point(color='Indigo').encode(
    x=alt.X('Genres:N', axis=alt.Axis(labelAngle=-45)),
    y=alt.Y('Categories:N'),
    size=alt.Size('sum(SteamSpyOwners):Q', scale=alt.Scale(range=[1, 700 * SCALE])),
    tooltip=alt.Tooltip(['sum(SteamSpyOwners):Q'])
).properties(
    width=500 * SCALE,
    height=250 * SCALE,
)

category_owner_bars = alt.Chart(category_exploded_df).transform_filter(
    alt.datum.Categories != 'nan'
).mark_bar(color='Indigo').encode(
    x=alt.X('sum(SteamSpyOwners):Q'),
    y=alt.Y('Categories:N', axis=None),
    tooltip=alt.Tooltip(['sum(SteamSpyOwners):Q'])
).properties(
    width=200 * SCALE,
    height=250 * SCALE
)

genre_owner_bars = alt.Chart(genre_exploded_df).transform_filter(
    alt.datum.Genres != 'nan'
).mark_bar(color='Indigo').encode(
    x=alt.X('Genres:N', axis=None),
    y=alt.Y('sum(SteamSpyOwners):Q'),
    tooltip=alt.Tooltip(['sum(SteamSpyOwners):Q'])
).properties(
    width=500 * SCALE,
    height=200 * SCALE
)

(genre_owner_bars & (category_genre_owner_chart | category_owner_bars)).properties(
    title='Owner count by category and genre',
).configure_title(
    fontSize=20,
)

alt.VConcatChart(...)

### Conclusion:


## **Price vs owner count:**
### Log axis ticks (haven't figured out how to make those work)

In [135]:
# x_log_ticks = np.arange(2.7, 7.5, .2)
# y_log_ticks = np.arange(-0.4, 2.8, .2)

# x_ticks = [10 ** xt for xt in x_log_ticks]
# y_ticks = [10 ** yt for yt in y_log_ticks]

alt.Chart(df).transform_filter(
    (alt.datum.PriceInitial > 0) & (alt.datum.SteamSpyOwners > 0)
).transform_calculate(
    LogOwners = 'log(datum.SteamSpyOwners)/log(10)',
    LogPrice = 'log(datum.PriceInitial)/log(10)'
).mark_rect().encode(
    x=alt.X(
        'LogOwners:Q',
        bin=alt.Bin(maxbins=100),

    ),
    y=alt.Y(
        'LogPrice:Q', 
        bin=alt.Bin(maxbins=40),

    ),
    color=alt.Color('count():Q', scale=alt.Scale(scheme='inferno'), sort='descending'),
).properties(
    width=800 * SCALE,
    title='Price vs. number of owners (Log scale)',
).configure_title(
    fontSize=20,
)

alt.Chart(...)

### Conclusion:
This visualization is a bit broken, as I haven't found a way to show proper axis ticks with log-transformed data.
However, using a good old calculator, we can see that most owned games cost either 4-5 USD or 9-11 USD

## **Owner count vs. Concurrent Player count:**

In [98]:
owner_player_scatter = alt.Chart(df).transform_filter(
    (alt.datum.SteamSpyOwners > 0) & (alt.datum.ConcurrentPlayers > 0)
).mark_circle().encode(
    x=alt.X(
        'ConcurrentPlayers:Q',
        scale=alt.Scale(type='log', base=10)
    ),
    y=alt.Y(
        'SteamSpyOwners:Q',
        scale=alt.Scale(type='log', base=10)
    ),
    color=alt.Color('DaysSinceRelease:Q', scale=alt.Scale(scheme='inferno'), sort='descending')
).properties(
    width=800 * SCALE,
    height=500 * SCALE,
    title='Owner count vs. Concurrent Player count (Log scale)',
).configure_title(
    fontSize=20,
).configure_mark(
     opacity = 1
)

owner_player_scatter

alt.Chart(...)

### Conclusion:
This graph is especially interesting, as in shows that there is basically no correlation between the number of owners and number of cocurrent players, even when accounting for retention. However, it shows that retention probably isn't random - we'll study that later.

## **Mean Metacritic score by genre:**

In [140]:
metacritic_heat = alt.Chart(category_genre_exploded_df).transform_filter(
    (alt.datum.Genres != 'nan') & (alt.datum.Categories != 'nan')
).mark_rect().encode(
    x=alt.X(
        'Genres:N',
        axis=alt.Axis(labelAngle=-45),
    ),
    y=alt.Y(
        'Categories:N'
    ),
    color=alt.Color('mean(Metacritic):Q', scale=alt.Scale(scheme='inferno'), sort='descending'),
    tooltip=alt.Tooltip(['mean(Metacritic):Q', 'sum(SteamSpyOwners)'])
).properties(
    width=700 * SCALE
)

metacritic_text = metacritic_heat.mark_text(
    align='center',
    baseline='middle',
).encode(
    text='count()',
    color=alt.value('LimeGreen')
).properties(
    width=700 * SCALE
)

metacritic_heat + metacritic_text

alt.LayerChart(...)

## **Metacritic score distribution:**

In [143]:
alt.Chart(df).transform_density(
    density='Metacritic',
    counts = True,
    steps=200
).mark_area(color='Indigo').encode(
    x=alt.X('value:Q'),
    y=alt.Y('density:Q'),
#     color=alt.Color('value:Q')
).properties(
    width=800 * SCALE,
    title='Metacritic score distribution',
).configure_title(
    fontSize=20,
).configure_mark(
     opacity = 1
)

alt.Chart(...)

## **Average retention curve: (TODO: Add recent data to study the trend)**

In [129]:
for r in range(len(df.index)):
    if df['SteamSpyOwners'][r] != 0 and (df['SteamSpyOwners'][r] < df['ConcurrentPlayers'][r]):
        print(df.iloc[r]['QueryName'])
        print(df.iloc[r]['SteamSpyOwners'])
        print(df.iloc[r]['ConcurrentPlayers'])
        

Sniper Elite 4
2511
2856
ELEX
803
922
Oxygen Not Included
533
7598
Totally Accurate Battle Simulator
535
2606
Wallpaper Engine
9326
15312
SCUM
535
4954
theHunter™: Call of the Wild
1600
4489


In [134]:
alt.Chart(df).transform_filter(
    (alt.datum.ConcurrentPlayers > 0) & (alt.datum.SteamSpyOwners > 0) & (alt.datum.SteamSpyOwners > alt.datum.ConcurrentPlayers)
).transform_calculate(
    Retention = 'datum.ConcurrentPlayers / datum.SteamSpyOwners'
).mark_point(color='Indigo', opacity=.1).encode(
    y=alt.Y('Retention:Q', scale=alt.Scale(type='log', base=2.7)),
    x=alt.X('DaysSinceRelease:Q'),
).properties(
    width=800 * SCALE
)

alt.Chart(...)

## **Correlation between score and player count:**

## **Correlation between price and owner count:**